# Superface Hub API - Anthropic Claude3 Example

This notebook demonstrates how to use Superface's Hub API to with the function calling capability of the Claude3 LLMs.

In [2]:
%pip install anthropic --quiet

## Setup and configuration

In [3]:
import anthropic
import json
import requests as r
from IPython.display import display, Markdown

# Set a random number of your choice, but don't change it
# once you have run the notebook, otherwise you will create another user.
SUPERFACE_USER_ID_CONSTANT = 123456789

# Use the number to create a unique ID
SUPERFACE_USER_ID = "sfoaidemo|" + str(SUPERFACE_USER_ID_CONSTANT)

# Default URL for Superface
SUPERFACE_BASE_URL = "https://pod.superface.ai/api/hub"

# Set the Superface authentication token
SUPERFACE_AUTH_TOKEN="<your-superface-api-token>"

# Set the OpenAI API Key
ANTHROPIC_API_KEY="<your-anthropic-api-key>"

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

## Helper functions
Defines helpers for communicating with the Superface HUB API, and also with Claude.

Anthropic expects a slightly different JSON schema for function definition than that used by OpenAI, MistralAI and LangChain, which is what Hub API delivers by default.

To combat this, the `get_formatted_tools()` function is required to alter the structure and keep Claude happy.

In [6]:
# Helper function to return the tool function descriptors
def get_superface_tools():
  headers = {"Authorization": "Bearer "+ SUPERFACE_AUTH_TOKEN}
  tools = r.get(SUPERFACE_BASE_URL + "/fd", headers=headers)
  return tools.json()

# Helper function to perform the action for all the functions.
# This is the only API call required regardless of what the function is.
def perform_action(tool_name=None, tool_body=None):
  headers = {"Authorization": "Bearer "+ SUPERFACE_AUTH_TOKEN, "x-superface-user-id": SUPERFACE_USER_ID}
  perform = r.post(SUPERFACE_BASE_URL + "/perform/" + tool_name, headers=headers, json=tool_body)
  return json.dumps(perform.json())

# Anthropic uses a slightly different schema, so reformat the function definitions JSON to suit
def get_formatted_tools():
  original_tools = get_superface_tools()
  formatted_tools = []

  for tool in original_tools:
    formatted_tools.append(tool['function'])

  for tool in formatted_tools:
    tool['input_schema'] = tool.pop("parameters")

  return formatted_tools

def talk_to_claude(role=None, message=None):
  messages.append({"role": role, "content": message})
  response = client.beta.tools.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    system="Today is April 5, 2024",
    tools=get_formatted_tools(),
    messages=messages,
  )
  return response

## Message history
The cell below sets up the message history. If you want to start your session again, you can just re-run this cell. There is no need to start again from the top of the notebook.

In [7]:
messages = []

## Submit an initial prompt

In [8]:
response = talk_to_claude("user", "What's the weather like in Prague?")
print(response)

ToolsBetaMessage(id='msg_01KL4PHnAeMiY13AaV3fLMQN', content=[TextBlock(text='<thinking>\nTo get the current weather forecast for Prague, I should use the weather__current-weather__CurrentWeather function. Let\'s check if I have the required parameters:\n\ncity: The user provided "Prague" as the city. To be more precise, I\'ll specify "Prague, Czech Republic".\nunits: This is an optional parameter. The user did not specify units, so I can omit this and the function will use the default of Celsius.\n\nI have the required city parameter, so I can proceed with calling the function.\n</thinking>', type='text'), ToolUseBlock(id='toolu_01SBw1Bov8VnjnL7Q6u16hYG', input={'city': 'Prague, Czech Republic'}, name='weather__current-weather__CurrentWeather', type='tool_use')], model='claude-3-opus-20240229', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=4595, output_tokens=179))


## Find out what Claude is thinking

In [9]:
print(response.content[0].text)

<thinking>
To get the current weather forecast for Prague, I should use the weather__current-weather__CurrentWeather function. Let's check if I have the required parameters:

city: The user provided "Prague" as the city. To be more precise, I'll specify "Prague, Czech Republic".
units: This is an optional parameter. The user did not specify units, so I can omit this and the function will use the default of Celsius.

I have the required city parameter, so I can proceed with calling the function.
</thinking>


## Run the tool Claude chose with Hub API

In [10]:
if (response.content[1] and response.content[1].type == "tool_use"):
  claude_response = response.content[1]
  messages.append({
      "role": "assistant",
      "content": [
          {
              "type": "text",
              "text": response.content[0].text
          },
          {
              "type": claude_response.type,
              "id": claude_response.id,
              "name": claude_response.name,
              "input": claude_response.input
          }
      ]
  })

  function_name = claude_response.name
  function_inputs = claude_response.input
  tool_use_id = claude_response.id

  superface_response = perform_action(function_name, function_inputs)

superface_response

'{"status": "success", "assistant_hint": "Format the result in \'result\' field to the user. If the user asked for a specific format, respect it", "result": {"description": "Partly cloudy", "feelsLike": 16, "temperature": 16}}'

In [11]:
tool_response_content = [{
    "type": "tool_result",
    "tool_use_id": tool_use_id,
    "content": superface_response

}]
claude_response = talk_to_claude("user", tool_response_content)
messages.append({"role": "assistant", "content": claude_response.content[0].text})

## Final response

In [12]:
display(Markdown(claude_response.content[0].text))

Current weather in Prague, Czech Republic:
Temperature: 16°C
Feels like: 16°C
Description: Partly cloudy